# Imports

In [1]:
use_kaggle=False
use_jupyter=True

In [2]:
if use_jupyter:
    print("Using Jupyter")
    !dir "C:/develop/python/Final Project/Data/"
elif use_kaggle:
    print("Using Kaggle")
    !ls "/kaggle/"
else:
    print("Using Colab with drive")
    from google.colab import drive
    drive.mount('/content/drive')
    !ls "/content/drive/My Drive/Colab Notebooks/ML course/"

Using Jupyter
 Volume in drive C has no label.
 Volume Serial Number is B0A9-3BF3

 Directory of C:\develop\python\Final Project\Data

09/04/2020  13:50    <DIR>          .
09/04/2020  13:50    <DIR>          ..
09/04/2020  11:29         9,955,043 Data.zip
07/04/2020  16:24    <DIR>          extracted-masks-images
07/04/2020  16:24         1,856,537 extracted_cells.csv
07/04/2020  15:43    <DIR>          masks-images
07/04/2020  15:43    <DIR>          resized-images
               2 File(s)     11,811,580 bytes
               5 Dir(s)  446,589,751,296 bytes free


In [3]:
import numpy as np
import pandas as pd
import uuid
import os
import csv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
import cv2
from skimage import io
from skimage import data, color, img_as_ubyte
from skimage.feature import canny
from skimage.transform import hough_ellipse
from skimage.draw import ellipse_perimeter, ellipse

# Consts and Setups

In [4]:
if use_jupyter:
    root_input_dir = 'C:/develop/python/Final Project/Data/'
    root_output_dir = 'C:/develop/python/Final Project/Data/'
elif use_kaggle:
    root_input_dir = '/kaggle/input/'
    root_output_dir = '/kaggle/working/output/'
else:
    root_input_dir = '/content/drive/My Drive/Colab Notebooks/ML course/Final Project - BioCell/'
    root_output_dir = '/content/drive/My Drive/Colab Notebooks/ML course/Final Project - BioCell/'

In [5]:
cell_masks_subdirectory = 'extracted-masks-images'
minimum_cover_percent = 89
max_factor_size = 4
uuid_row_index = 1
input_csv_filename = 'extracted_cells.csv'
output_csv_filename = 'filtered_cells.csv'
images_extension = ".png"
csv_field_names = ['source_file', 'mask_uuid', 'left_x', 'right_x', 'top_y', 'bottom_y', 'id']
start_line=0
end_line=5000

In [6]:
plt.gray()
np.seterr(divide='ignore', invalid='ignore')

# Inputs
input_csv_file_path = root_input_dir + input_csv_filename
masks_images_path = root_input_dir + cell_masks_subdirectory + '/'
input_csv_rows_list = []

# Outputs
output_csv_rows_list = []
output_csv_file_path = root_output_dir + f'{start_line}-{end_line}_' + output_csv_filename

<Figure size 432x288 with 0 Axes>

# Function Definition

In [7]:
def getMaskBestEllipse(mask_data):
    edges = canny(mask_data, sigma=2.0, low_threshold=0.55, high_threshold=0.8)
    ellipses_data = hough_ellipse(edges, accuracy=10, threshold=40, min_size=0, max_size=500)
    ellipses_data.sort(order='accumulator')
    best = list(ellipses_data[-1])
    yc, xc, a, b = [int(round(x)) for x in best[1:5]]
    orientation = best[5]
    return yc, xc, a, b, orientation

In [8]:
def resizeImage(image_data, scale_size):
    width = int(current_mask_data.shape[1] * scale_size)
    height = int(current_mask_data.shape[0] * scale_size)
    dim = (width, height)
    # resize image
    return cv2.resize(current_mask_data, dim, interpolation = cv2.INTER_AREA)

In [9]:
def getEllipseCoverPercent(mask_data, factor_size):
    # Resize the image first
    resized_image = resizeImage(mask_data, factor_size)
    mask_copy = resized_image.copy()

    # Get the best ellipse that covers the mask
    yc, xc, a, b, orientation = getMaskBestEllipse(resized_image)

    # Try to draw the best ellipse on the mask
    try:
        # Get the indices of the ellipse fill and the ellipse perimiter
        perimiter_y, perimiter_x = ellipse_perimeter(yc, xc, a, b, orientation)
        fill_y, fill_x = ellipse(yc, xc, a, b, rotation=-orientation)

        # Set 0 (black) in the mask copy where the indexes are located
        mask_copy[list(perimiter_y), list(perimiter_x)] = 0
        mask_copy[list(fill_y), list(fill_x)] = 0
        
        # Get the number of nonzero pixels - that is the number of mask pixels that the ellipse does not cover
        full_mask_nonzero = np.count_nonzero(resized_image)
        ellipse_non_zero = np.count_nonzero(mask_copy)

        return ((full_mask_nonzero - ellipse_non_zero) / full_mask_nonzero) * 100
    except:
        print(f'Could not get ellipse for factor size {factor_size}')
        return 0

# Filter Logic

In [10]:
# Read the csv file, which is the output of the extraction module
# Loop over the lines and foreach line extract and read the mask image (using the uuid)
# get the ellipse data from the getMaskBestEllipse function
# set the mask copy values to 0 (black) in the ellipse area
# calculate the number of nonzero pixels in the mask copy
# if it exceeds a predefined value (max_white_pixel_outside_ellipse, need to decide what is the best value) - it's not a good filter
# if it does not exceed - a good filter, insert the line from the input csv to a new output csv

with open(input_csv_file_path) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    input_csv_rows_list = list(csv_reader)
    counter = start_line
    for line in input_csv_rows_list[start_line:end_line + 1]:
        start = time.time()
        counter = counter + 1
        # Get the mask data
        current_mask_uuid = line[uuid_row_index]
        current_mask_filename = current_mask_uuid + images_extension
        current_mask_data = io.imread(masks_images_path + current_mask_filename, as_gray=True)

        factor_size = 2
        ellipse_detected = False
        
        while factor_size <= max_factor_size and not ellipse_detected:
            # Get the coverage percent for the curretn sace factor
            current_cover_percent = getEllipseCoverPercent(current_mask_data, factor_size)

            # If the coverage percentage is greater than the wanted minimum - this is a good mask
            if current_cover_percent > minimum_cover_percent:
                ellipse_detected = True
                output_csv_rows_list.append({
                    "source_file": line[0],
                    "mask_uuid": line[1],
                    "left_x": line[2],
                    "right_x": line[3],
                    "top_y": line[4],
                    "bottom_y": line[5],
                    "id": line[6]
                })
            
            factor_size = factor_size + 1
        end = time.time()
        print(f'line number {counter} - time {end - start} seconds')
                    
                    
with open(output_csv_file_path, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=csv_field_names)
    print(f'Writing {len(output_csv_rows_list)} filtered lines to {output_csv_filename}')
    writer.writerows(output_csv_rows_list)

line number 1 - time 0.3672060966491699 seconds
line number 2 - time 6.893190383911133 seconds
line number 3 - time 2.5903079509735107 seconds
line number 4 - time 2.3877668380737305 seconds
line number 5 - time 4.774552345275879 seconds
line number 6 - time 2.057337522506714 seconds
line number 7 - time 3.318013906478882 seconds
line number 8 - time 2.363651752471924 seconds
line number 9 - time 3.6594784259796143 seconds
line number 10 - time 2.964163303375244 seconds
line number 11 - time 0.38028454780578613 seconds
line number 12 - time 0.5577728748321533 seconds
line number 13 - time 4.040472984313965 seconds
line number 14 - time 0.6977798938751221 seconds
line number 15 - time 0.6856892108917236 seconds
line number 16 - time 5.324412107467651 seconds
line number 17 - time 0.6141643524169922 seconds
line number 18 - time 0.5521137714385986 seconds
line number 19 - time 10.83114242553711 seconds
line number 20 - time 29.103675365447998 seconds
line number 21 - time 4.2114100456237

line number 167 - time 1.943260669708252 seconds
line number 168 - time 3.2102227210998535 seconds
line number 169 - time 0.5164873600006104 seconds
line number 170 - time 2.592421531677246 seconds
line number 171 - time 4.2695207595825195 seconds
line number 172 - time 2.086437463760376 seconds
line number 173 - time 1.7781689167022705 seconds
line number 174 - time 14.65582275390625 seconds
line number 175 - time 3.0103697776794434 seconds
line number 176 - time 7.078863620758057 seconds
line number 177 - time 3.137249708175659 seconds
line number 178 - time 9.663085222244263 seconds
line number 179 - time 3.8364498615264893 seconds
line number 180 - time 0.6328177452087402 seconds
line number 181 - time 7.268115997314453 seconds
line number 182 - time 2.0983991622924805 seconds
line number 183 - time 4.248042106628418 seconds
line number 184 - time 3.0671186447143555 seconds
line number 185 - time 4.066772937774658 seconds
line number 186 - time 2.029162883758545 seconds
line number

line number 330 - time 8.244784832000732 seconds
line number 331 - time 1.790259838104248 seconds
line number 332 - time 1.7519207000732422 seconds
line number 333 - time 0.5402777194976807 seconds
line number 334 - time 0.6211512088775635 seconds
line number 335 - time 2.5010335445404053 seconds
line number 336 - time 0.5020482540130615 seconds
line number 337 - time 0.5021791458129883 seconds
line number 338 - time 2.4354124069213867 seconds
line number 339 - time 2.4335227012634277 seconds
line number 340 - time 3.0222408771514893 seconds
line number 341 - time 3.0335633754730225 seconds
line number 342 - time 3.178950548171997 seconds
line number 343 - time 3.1031694412231445 seconds
line number 344 - time 2.8139736652374268 seconds
line number 345 - time 1.518549919128418 seconds
line number 346 - time 3.642033100128174 seconds
line number 347 - time 9.140400409698486 seconds
line number 348 - time 5.15704345703125 seconds
line number 349 - time 4.429949760437012 seconds
line numb

line number 494 - time 3.5849013328552246 seconds
line number 495 - time 2.0108819007873535 seconds
line number 496 - time 10.339024543762207 seconds
line number 497 - time 3.5261409282684326 seconds
line number 498 - time 3.695789337158203 seconds
line number 499 - time 2.5437984466552734 seconds
line number 500 - time 0.40747928619384766 seconds
line number 501 - time 3.857990264892578 seconds
line number 502 - time 2.6529123783111572 seconds
line number 503 - time 4.353213310241699 seconds
line number 504 - time 2.110807180404663 seconds
line number 505 - time 2.0464231967926025 seconds
line number 506 - time 3.4320626258850098 seconds
line number 507 - time 5.310096740722656 seconds
line number 508 - time 2.4569344520568848 seconds
line number 509 - time 1.33498215675354 seconds
line number 510 - time 11.807991027832031 seconds
line number 511 - time 3.113191843032837 seconds
line number 512 - time 3.4639267921447754 seconds
line number 513 - time 0.48851943016052246 seconds
line n

line number 658 - time 2.1270458698272705 seconds
line number 659 - time 4.077102184295654 seconds
line number 660 - time 27.134208917617798 seconds
line number 661 - time 2.796053886413574 seconds
line number 662 - time 7.412622690200806 seconds
line number 663 - time 4.963374137878418 seconds
line number 664 - time 11.006553173065186 seconds
line number 665 - time 5.829050540924072 seconds
line number 666 - time 10.157465696334839 seconds
line number 667 - time 0.500939130783081 seconds
line number 668 - time 3.9979946613311768 seconds
line number 669 - time 0.5028576850891113 seconds
line number 670 - time 1.9230334758758545 seconds
line number 671 - time 0.4800229072570801 seconds
line number 672 - time 2.429018497467041 seconds
line number 673 - time 2.225813865661621 seconds
line number 674 - time 5.161254405975342 seconds
line number 675 - time 2.4522311687469482 seconds
line number 676 - time 1.9948513507843018 seconds
line number 677 - time 6.8139564990997314 seconds
line numb

...

line number 4203 - time 14.867340803146362 seconds
line number 4204 - time 4.842877388000488 seconds
line number 4205 - time 3.1708855628967285 seconds
line number 4206 - time 3.8333630561828613 seconds
line number 4207 - time 2.368168830871582 seconds
line number 4208 - time 0.5532805919647217 seconds
line number 4209 - time 0.9993014335632324 seconds
line number 4210 - time 0.550809383392334 seconds
line number 4211 - time 1.8215484619140625 seconds
line number 4212 - time 1.3442463874816895 seconds
line number 4213 - time 0.5065205097198486 seconds
line number 4214 - time 2.335834503173828 seconds
line number 4215 - time 3.3711562156677246 seconds
line number 4216 - time 1.4777004718780518 seconds
line number 4217 - time 3.0255043506622314 seconds
line number 4218 - time 3.644836664199829 seconds
line number 4219 - time 0.6167776584625244 seconds
line number 4220 - time 3.3007378578186035 seconds
line number 4221 - time 0.5386333465576172 seconds
line number 4222 - time 3.0663845539

line number 4364 - time 23.831080436706543 seconds
line number 4365 - time 4.921135902404785 seconds
line number 4366 - time 0.504727840423584 seconds
line number 4367 - time 2.331165075302124 seconds
line number 4368 - time 0.4321451187133789 seconds
line number 4369 - time 3.2823870182037354 seconds
line number 4370 - time 5.488687038421631 seconds
line number 4371 - time 0.5136294364929199 seconds
line number 4372 - time 1.7770030498504639 seconds
line number 4373 - time 4.765821695327759 seconds
line number 4374 - time 0.5434010028839111 seconds
line number 4375 - time 8.387847900390625 seconds
line number 4376 - time 0.5268065929412842 seconds
line number 4377 - time 5.554872751235962 seconds
line number 4378 - time 7.990670442581177 seconds
line number 4379 - time 2.283672571182251 seconds
line number 4380 - time 3.0983126163482666 seconds
line number 4381 - time 2.839329242706299 seconds
line number 4382 - time 4.747812032699585 seconds
line number 4383 - time 2.993406295776367 

line number 4525 - time 1.738431692123413 seconds
line number 4526 - time 4.053893327713013 seconds
line number 4527 - time 2.3120381832122803 seconds
line number 4528 - time 0.4745817184448242 seconds
line number 4529 - time 0.5462648868560791 seconds
line number 4530 - time 0.8109836578369141 seconds
line number 4531 - time 12.425684213638306 seconds
line number 4532 - time 3.791386842727661 seconds
line number 4533 - time 4.761124134063721 seconds
line number 4534 - time 0.5766232013702393 seconds
line number 4535 - time 0.4155597686767578 seconds
line number 4536 - time 2.3655943870544434 seconds
line number 4537 - time 9.86649775505066 seconds
line number 4538 - time 2.1269750595092773 seconds
line number 4539 - time 3.6669352054595947 seconds
line number 4540 - time 0.49637770652770996 seconds
line number 4541 - time 2.034348249435425 seconds
line number 4542 - time 1.860076665878296 seconds
line number 4543 - time 0.5226607322692871 seconds
line number 4544 - time 2.251832723617

Could not get ellipse for factor size 2
Could not get ellipse for factor size 3
line number 4687 - time 4.133040428161621 seconds
line number 4688 - time 0.5139679908752441 seconds
line number 4689 - time 0.48885416984558105 seconds
line number 4690 - time 3.0211853981018066 seconds
line number 4691 - time 0.6357605457305908 seconds
line number 4692 - time 3.7880022525787354 seconds
line number 4693 - time 2.925774335861206 seconds
line number 4694 - time 0.46952390670776367 seconds
line number 4695 - time 15.595711946487427 seconds
line number 4696 - time 3.7593069076538086 seconds
line number 4697 - time 4.957652568817139 seconds
line number 4698 - time 8.403885126113892 seconds
line number 4699 - time 2.2655491828918457 seconds
line number 4700 - time 9.649239301681519 seconds
line number 4701 - time 2.016390085220337 seconds
line number 4702 - time 1.1127045154571533 seconds
line number 4703 - time 0.4420011043548584 seconds
line number 4704 - time 2.2542481422424316 seconds
line n

line number 4847 - time 0.49941444396972656 seconds
line number 4848 - time 2.636138439178467 seconds
line number 4849 - time 0.4164888858795166 seconds
line number 4850 - time 0.9625895023345947 seconds
line number 4851 - time 0.8151538372039795 seconds
line number 4852 - time 8.305175304412842 seconds
line number 4853 - time 0.4581897258758545 seconds
line number 4854 - time 4.032211065292358 seconds
line number 4855 - time 0.6199026107788086 seconds
line number 4856 - time 5.578470945358276 seconds
line number 4857 - time 0.3323359489440918 seconds
line number 4858 - time 4.875468969345093 seconds
line number 4859 - time 1.5588715076446533 seconds
line number 4860 - time 3.4467523097991943 seconds
line number 4861 - time 5.25011682510376 seconds
line number 4862 - time 5.244759798049927 seconds
line number 4863 - time 4.566465854644775 seconds
line number 4864 - time 3.043769598007202 seconds
line number 4865 - time 2.8633594512939453 seconds
line number 4866 - time 0.58295702934265